# Imports

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
from sklearn.preprocessing import StandardScaler

# Data loading

In [ ]:
from sklearn.datasets import make_classification
X,y = make_classification(n_features=10, n_informative=6, n_classes=3, n_samples=10000)

`Read the data into X and y numpy arrays`

# Data preparation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)   #stratify = y

s = StandardScaler().fit(X_train)
X_train = s.fit_transform(X_train)
X_test = s.fit_transform(X_test)


# X_train = X_train.reshape(-1,1)
# X_test = X_test.reshape(-1,1)


X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


tensor_train = TensorDataset(X_train, y_train)
#tensor_test = TensorDataset(X_test, y_test)

training_dataset = DataLoader(tensor_train, batch_size = 32, shuffle=True)

# Creating the deep learning model

In [ ]:
class mod(nn.Module):

    def __init__(self):
        
        super(mod, self).__init__()

        self.h1 = nn.Linear(in_features=10, out_features=15)

        self.bnn = nn.BatchNorm1d(num_features=15)
        self.hn = nn.Linear(in_features=15, out_features=3)
        
        self.drop = nn.Dropout()

    def forward(self, x):

        x = self.h1(x)
        x = F.relu(x)
        x = self.drop(x)
        
        #x = self.bnn(x)
        x = self.hn(x)

        return x

# Running the model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = mod()
model = model.to(device)

opt = Adam(params = model.parameters())

n_epochs = 1000

for epoch in range(n_epochs):
    training_loss = 0.0
    
    for batch, target in training_dataset:

        batch = batch.float().to(device)
        target = target.long().to(device)

        batch_prediction = model(batch)

        loss = F.cross_entropy(batch_prediction,target) # loss = F.mse_loss(target, batch_prediction)
        training_loss += loss.item()

        loss.backward()
        opt.step()
        opt.zero_grad()
    #yp = test_prediction.argmax(axis=1)
    test_prediction = model(X_test.float().to(device))
    test_loss = F.cross_entropy(test_prediction, y_test.long()) # loss = F.mse_loss(y_test, test_prediction)

    if epoch % 50 == 0:
        print(f"Training loss: {training_loss}, test loss: {test_loss}")


Training loss: 252.8471086025238, test loss: 0.9314826726913452
Training loss: 173.05776444077492, test loss: 0.6936585307121277
Training loss: 170.0288051366806, test loss: 0.7060844898223877
Training loss: 170.3834564089775, test loss: 0.6797645092010498
Training loss: 170.21337622404099, test loss: 0.7163775563240051
Training loss: 170.08115217089653, test loss: 0.693203866481781
Training loss: 170.69442000985146, test loss: 0.6782982349395752
Training loss: 170.5862332880497, test loss: 0.6778146624565125
Training loss: 171.43888267874718, test loss: 0.6840017437934875
Training loss: 169.52847349643707, test loss: 0.7069750428199768
Training loss: 170.79129886627197, test loss: 0.6956117153167725
Training loss: 171.67725339531898, test loss: 0.6897947192192078
Training loss: 167.53862020373344, test loss: 0.6736897826194763
Training loss: 170.26278534531593, test loss: 0.6961482763290405
Training loss: 168.73461318016052, test loss: 0.6897329688072205
Training loss: 170.01959913969